In [1]:
from bs4 import BeautifulSoup
import re
import requests
import pandas as pd

In [2]:
entry_urls = []

start_date = "2019-01-01"
end_date = "today"
base_url = "https://rip.ie/death-notice/s/all?page={}&start={}%2B00%3A00%3A00&end={}&sortField=a.createdAtCastToDate&sortDir=DESC&view=boxes"


for i in range(0, 3):
    entry_urls.append(base_url.format(i, start_date, end_date))

In [3]:
def scrape(html):
    soup = BeautifulSoup(html, 'html.parser')
    return soup

In [4]:
def scrape_url(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            return response.text
        else:
            print(f"Failed to retrieve data from {url}. Status code: {response.status_code}")
            return None
    except Exception as e:
        print(f"Error while processing {url}: {str(e)}")
        return None

In [5]:
def get_url(soup):
    url_list = []
    anchors = soup.find_all('a', class_='showdown-dn-link')
    urls = [a['href'] for a in anchors]
    for url in urls:
        url_list.append("https://rip.ie" + url)
    return url_list

In [6]:
def get_individual_date(soup):
    date_list = []
    published_date_element = soup.find('p', class_='DeathNotice_dates-published-date__M0A_i')
    death_date_element = soup.find('p', class_='DeathNotice_dates-death-date__bR7g_')
    published_date = published_date_element.get_text()
    death_date = death_date_element.get_text()
    date_list.append(published_date)
    date_list.append(death_date)
    return date_list

In [7]:
def get_page_data(entry_urls):
    data_list = []
    
    url_pattern = r'https://rip.ie/death-notice/([^/]+)-([^/]+)-([^/]+)(?:-([^/]+))?-(\d+)'
    
    for entry_url in entry_urls:
        html_data = scrape_url(entry_url)
        
        if html_data:
            url_list = get_url(scrape(html_data))
            
            for u in url_list:
                match = re.match(url_pattern, u)

                if match:
                    name = match.group(1)
                    city = match.group(2)
                    location = match.group(3)
                    identity = match.group(5)
                    
                    dates = get_individual_date(scrape(html_data))

                    data_list.append({
                                    "Name": name,
                                    "ID": identity,
                                    "City": city,
                                    "Location": location,
                                    "Published Date": dates[0],
                                    "Date": dates[1]
                                })

                else:
                    print("Pattern did not match.")
                    
    df = pd.DataFrame(data_list)
    return df

In [8]:
get_page_data(entry_urls)

AttributeError: 'NoneType' object has no attribute 'get_text'